**Summary:**

This notebook shows how to train a Bert model for the RACE dataset by using the Huggingface *transformers* and *datasets* libraries.

In [ ]:
!nvidia-smi

Mon Nov  9 16:28:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Clone and install**

In [ ]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 50268 (delta 45), reused 71 (delta 11), pack-reused 50154
Receiving objects: 100% (50268/50268), 37.11 MiB | 30.50 MiB/s, done.
Resolving deltas: 100% (35043/35043), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 3.5MB/s 
     |████████████████████████████████| 2.9MB 17.6MB/s 
     |████████████████████████████████| 1.1MB 40.4MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1297777 sha256=b0ba9cabe278b86895d7adb9fc81a60e66510874b1dedd0456202beac43b1971
  Stored in directory: /tmp/pip-ephem-wheel-cache-_tnjnag7/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Success

/content


In [ ]:
!pip install datasets

     |████████████████████████████████| 17.7MB 212kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
cd transformers/examples/multiple-choice/

/content/transformers/examples/multiple-choice


**Generic imports**

In [ ]:
import logging
import os

import numpy as np

**Huggingface imports**

In [ ]:
from datasets import load_dataset

from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)

from utils_multiple_choice import *

In [ ]:
cd ../../..

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Load dataset and model**

In [ ]:
dataset = load_dataset("race", "middle")
model = AutoModelForMultipleChoice.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Dataset race downloaded and prepared to /root/.cache/huggingface/datasets/race/middle/0.1.0/a7d1fac780e70c0e75bca35e9f2f8cfc1411edd18ffd6858ddce56f70dfb1e7c. Subsequent calls will reuse this data.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

**Training args**

In [ ]:
label_list = ["0", "1", "2", "3"]
max_seq_length = 468

batch_size = 4
training_args = TrainingArguments(
  "test-multichoice",
  evaluation_strategy = "epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  num_train_epochs=3,
  weight_decay=0.01,
  load_best_model_at_end=True,
)

**Create examples from HuggingFace Dataset**

In [ ]:
train_examples = []
data_train = dataset['train']
for ex in data_train:
    race_id = ex["example_id"]
    article = ex["article"]

    truth = str(ord(ex["answer"]) - ord("A"))
    question = ex["question"]
    options = ex["options"]

    train_examples.append(
      InputExample(
          example_id=race_id,
          question=question,
          contexts=[article, article, article, article],  # this is not efficient but convenient
          endings=[options[0], options[1], options[2], options[3]],
          label=truth,
      )
    )

val_examples = []
data_val = dataset['validation']
for ex in data_val:
    race_id = ex["example_id"]
    article = ex["article"]

    truth = str(ord(ex["answer"]) - ord("A"))
    question = ex["question"]
    options = ex["options"]

    val_examples.append(
      InputExample(
          example_id=race_id,
          question=question,
          contexts=[article, article, article, article],  # this is not efficient but convenient
          endings=[options[0], options[1], options[2], options[3]],
          label=truth,
      )
    )

**Convert examples to features**

In [ ]:
train_features = convert_examples_to_features(
    train_examples,
    label_list,
    max_seq_length,
    tokenizer,
)

val_features = convert_examples_to_features(
    val_examples,
    label_list,
    max_seq_length,
    tokenizer,
)

convert examples to features: 25421it [06:54, 61.37it/s]
convert examples to features: 1436it [00:23, 61.29it/s]


**Create trainer**

In [ ]:
def simple_accuracy(preds, labels):
    return (preds == labels).mean()
    
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": simple_accuracy(preds, p.label_ids)}

# Initialize our Trainerlo
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_features,
  eval_dataset=val_features,
  compute_metrics=compute_metrics,
)

**Train and eval**

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Acc
1,0.981756,0.939171,0.641365
2,0.716012,1.012905,0.661560
3,0.455713,1.430195,0.675487


TrainOutput(global_step=19068, training_loss=0.7554399263491189)

In [ ]:
trainer.evaluate()

{'epoch': 3.0, 'eval_acc': 0.6413649025069638, 'eval_loss': 0.9391707181930542}

**Save model**

In [ ]:
try:
    trainer.save_model("drive/My Drive/bbu_race_m/")
    tokenizer.save_pretrained("drive/My Drive/bbu_race_m/")
except:
    trainer.save_model("bbu_race_m/")
    tokenizer.save_pretrained("bbu_race_m/")